# Je suis tous les conseils de GPT

In [ ]:
%pip install ydata-profiling

In [ ]:
import numpy as np
import pandas as pd
from scipy import stats
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.nn.functional as F
import re
from ydata_profiling import ProfileReport


train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
gender_submission = pd.read_csv('gender_submission.csv')


print("\nNombre de NaN par colonne pour chaque DataFrame:")
print("train:\n", train.isna().sum())
print("\ntest:\n", test.isna().sum())
print("\ngender_submission:\n", gender_submission.isna().sum())

print("\nTypes de chaque colonne pour chaque DataFrame:")
print("train:\n", train.dtypes)
print("\ntest:\n", test.dtypes)
print("\ngender_submission:\n", gender_submission.dtypes)




Nombre de NaN par colonne pour chaque DataFrame:
train:
 PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

test:
 PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64

gender_submission:
 PassengerId    0
Survived       0
dtype: int64

Types de chaque colonne pour chaque DataFrame:
train:
 PassengerId      int64
Survived         int64
Pclass           int64
Name            object
Sex             object
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Cabin           object
Embarked        object
dtype: object

test:
 PassengerId      int64
Pclass           int64
Nam

In [ ]:
train.head(20)

In [ ]:
# Manipulation des features
train["Age"] = train.groupby(["Sex", "Pclass"])["Age"].transform(
    lambda x: x.fillna(x.median())
)
test["Age"] = test.groupby(["Sex", "Pclass"])["Age"].transform(
    lambda x: x.fillna(x.median())
)
train['Jeune'] = (train['Age'] < 14).astype(int)
test['Jeune'] = (test['Age'] < 14).astype(int)

train['Vieux'] = (train['Age'] >= 60).astype(int)
test['Vieux'] = (test['Age'] >= 60).astype(int)

# Extraction du titre depuis la colonne Name
def extract_title(name):
    title_search = re.search(r',\s*([^\.]*)\.', name)
    if title_search:
        title = title_search.group(1).strip()
        if title in ['Mr', 'Mrs', 'Miss', 'Master']:
            return title
        else:
            return 'Rare'
    return 'Rare'

train['Title'] = train['Name'].apply(extract_title)
test['Title'] = test['Name'].apply(extract_title)

mapping = {'Mr': 0, 'Miss': 1, 'Mrs': 2, 'Master': 3, 'Rare': 4}
train['Title'] = train['Title'].map(mapping)
test['Title'] = test['Title'].map(mapping)

# Taille de la famille et personne seule
train['FamilySize'] = train['SibSp'] + train['Parch'] + 1
test['FamilySize'] = test['SibSp'] + test['Parch'] + 1

train['isAlone'] = (train['FamilySize'] == 1).astype(int)
test['isAlone'] = (test['FamilySize'] == 1).astype(int)

# Suppression des colonnes inutiles
train = train.drop(columns=['Flare' ,'Ticket','Name','Age','Pclass','SibSp','Parch'], errors='ignore')
test = test.drop(columns=['Flare','Ticket','Name','Age','Pclass','SibSp','Parch'], errors='ignore')


# Création des colonnes indicatrices pour Pclass
for i in [1, 2, 3]:
    train[f'Classe_{i}'] = (train['Pclass'] == i).astype(int)
    test[f'Classe_{i}'] = (test['Pclass'] == i).astype(int)


print(train['Sex'].value_counts())

train['Sex'] = train['Sex'].map({'male': -1, 'female': 1})
test['Sex'] = test['Sex'].map({'male': -1, 'female': 1})

print("\nNombre de NaN par colonne pour chaque DataFrame:")
print("train:\n", train.isna().sum())
print("\ntest:\n", test.isna().sum())

print("\nTypes de chaque colonne pour chaque DataFrame:")
print("train:\n", train.dtypes)
print("\ntest:\n", test.dtypes)